## Generate 3D Mock Map

1. Generate a 3d mesh object of an oil palm tree
2. Place at every location repeating it (use same mesh, repeat for every different seed)
3. Scale it according to age of oil palm tree
4. Voxelize and downsample it to .pcd

In [ ]:
%pip install open3d noise

In [ ]:
import open3d as o3d
print(hasattr(o3d, 'io'))

In [ ]:
crown_path = 'sample_data/crowns_exported.gpkg'
mesh_path = 'InstantMesh/outputs/instant-mesh-base/meshes/1.obj'
pcd_export_path = 'sample_data/pc_exported.pcd'
utm_epsg = "EPSG:32647"

seed_offset_x = 69
seed_offset_y = 42

In [ ]:
import open3d as o3d
import numpy as np
import copy
import geopandas as gpd
from shapely.geometry import shape
from noise import pnoise2

# --- Load and reproject crowns to UTM ---
def extract_centroids_from_geojson(geojson_path, utm_epsg):
    crowns = gpd.read_file(geojson_path)
    if crowns.crs != utm_epsg:
        crowns = crowns.to_crs(utm_epsg)
    centroids = crowns.geometry.centroid
    return [(pt.x, pt.y) for pt in centroids]

# --- Generate Perlin elevation ---
def get_perlin_elevation(x, y, scale=0.01, octaves=4, persistence=0.7, lacunarity=2.0, elevation_scale=5.0):
    return pnoise2((seed_offset_x + x) * scale, (seed_offset_y + y) * scale, octaves=octaves, persistence=persistence, lacunarity=lacunarity) * elevation_scale

# --- Generate terrain mesh with heatmap ---
def generate_terrain_mesh(centroid_coords, grid_spacing=1.0, grid_size=150, elevation_scale=3.0):
    xs, ys = zip(*centroid_coords)
    center_x, center_y = np.mean(xs), np.mean(ys)

    x_vals = np.arange(center_x - grid_size/2, center_x + grid_size/2, grid_spacing)
    y_vals = np.arange(center_y - grid_size/2, center_y + grid_size/2, grid_spacing)

    # Precompute z values and find min/max
    z_vals = [get_perlin_elevation(x, y, elevation_scale=elevation_scale)
              for y in y_vals for x in x_vals]
    z_min, z_max = min(z_vals), max(z_vals)

    def get_gradient_color(z, z_min, z_max):
        import matplotlib
        if z_max == z_min:  # Avoid division by zero
            return np.array([0.5, 0.5, 0.5])
        
        norm_z = (z - z_min) / (z_max - z_min)
        clamped_norm_z = np.clip(norm_z, 0, 1)

        cmap = matplotlib.colormaps.get_cmap('terrain') # gist_earth, terrain
        # Get RGB color from colormap, ignore Alpha channel
        color = cmap(clamped_norm_z)[:3] 
        return color

    vertices = []
    colors = []
    idx = 0
    for y in y_vals:
        for x in x_vals:
            z = z_vals[idx]
            idx += 1
            vertices.append([x, y, z])
            colors.append(get_gradient_color(z, z_min, z_max))

    vertices = np.array(vertices)
    n_x = len(x_vals)
    n_y = len(y_vals)
    triangles = []

    for j in range(n_y - 1):
        for i in range(n_x - 1):
            idx = j * n_x + i
            triangles.append([idx, idx + 1, idx + n_x])
            triangles.append([idx + 1, idx + n_x + 1, idx + n_x])

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    mesh.compute_vertex_normals()
    return mesh

centroid_coords = extract_centroids_from_geojson(crown_path, utm_epsg)
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()

scene_meshes = []
for x, y in centroid_coords:
    instance = copy.deepcopy(mesh)

    # Random scaling
    scale = np.random.uniform(4.75, 5.25)
    instance.scale(scale, center=instance.get_center())

    # Random yaw rotation
    yaw = np.random.uniform(0, 2 * np.pi)
    R = instance.get_rotation_matrix_from_xyz((np.pi, 0, yaw))
    instance.rotate(R, center=instance.get_center())

    # Elevation + offset so base aligns with terrain
    terrain_z = get_perlin_elevation(x, y)
    z_offset = -instance.get_min_bound()[2]
    instance.translate((x, y, terrain_z + z_offset))

    scene_meshes.append(instance)

# Combine palm meshes
scene = scene_meshes[0]
for inst in scene_meshes[1:]:
    scene += inst

# Convert the combined mesh to a point cloud
pcd_from_mesh = scene.sample_points_uniformly(number_of_points=500000)
# pcd_from_mesh = scene.sample_points_poisson_disk(number_of_points=500000, init_factor=5)

# Export the point cloud
o3d.io.write_point_cloud(pcd_export_path, pcd_from_mesh)
print(f"Exported combined mesh to PCD: {pcd_export_path} with {len(pcd_from_mesh.points)} points.")

# Voxelize
# voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(scene, voxel_size=1.0)

# Generate terrain mesh
terrain_mesh = generate_terrain_mesh(centroid_coords)

o3d.visualization.draw_geometries(
    [terrain_mesh, pcd_from_mesh], # pcd_from_mesh, voxel_grid
    window_name="Voxelized Scene with Terrain",
    width=1000,
    height=800
)